In [84]:
import nltk
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn_crfsuite as crfsuite
from sklearn_crfsuite import metrics
import os
from pathlib import Path
import re
import pickle
import random

NUM_JANELA=4

In [4]:
entidade = ['dor', 'no', 'braço', 'esquerdo','e','direito']

lista=list()
for i in range(1, len(entidade)+1):
    #lista.append(entidade[i-1])
    for j in range(len(entidade) - i + 1):
        lista.append(entidade[j:j + i])
        
lista

[['dor'],
 ['no'],
 ['braço'],
 ['esquerdo'],
 ['e'],
 ['direito'],
 ['dor', 'no'],
 ['no', 'braço'],
 ['braço', 'esquerdo'],
 ['esquerdo', 'e'],
 ['e', 'direito'],
 ['dor', 'no', 'braço'],
 ['no', 'braço', 'esquerdo'],
 ['braço', 'esquerdo', 'e'],
 ['esquerdo', 'e', 'direito'],
 ['dor', 'no', 'braço', 'esquerdo'],
 ['no', 'braço', 'esquerdo', 'e'],
 ['braço', 'esquerdo', 'e', 'direito'],
 ['dor', 'no', 'braço', 'esquerdo', 'e'],
 ['no', 'braço', 'esquerdo', 'e', 'direito'],
 ['dor', 'no', 'braço', 'esquerdo', 'e', 'direito']]

In [5]:
def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

def save_obj(name, obj):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    print('Load obj em: ', 'obj/' + name + '.pkl')
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [6]:
dic_sentencesTrain = load_obj(r'../spanclassification/obj/dic_sentencesTrain')
dic_sentencesDev = load_obj(r'../spanclassification/obj/dic_sentencesDev')
dic_sentencesTest = load_obj(r'../spanclassification/obj/dic_sentencesTest')


Load obj em:  obj/../spanclassification/obj/dic_sentencesTrain.pkl
Load obj em:  obj/../spanclassification/obj/dic_sentencesDev.pkl
Load obj em:  obj/../spanclassification/obj/dic_sentencesTest.pkl


In [17]:
dic_sentencesTest[1]

[[['Em', 0, 59],
  ['acompanhamento', 1, 62],
  ['no', 2, 77],
  ['ambualtorio', 3, 80],
  ['há', 4, 92],
  ['5', 5, 95],
  ['anos', 6, 97],
  ['por', 7, 102],
  ['FA', 8, 106],
  [',', 9, 108],
  ['uso', 10, 110],
  ['de', 11, 114],
  ['marevan', 12, 117],
  ['5mg', 13, 125],
  ['1', 14, 129],
  ['x', 15, 131],
  ['ao', 16, 133],
  ['dia', 17, 136],
  ['.', 18, 139]],
 [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]]

In [202]:
def getListaPositiva(dic_sentences):
    # só positivas:
    listaSentencas=list()
    lista=list()
    numJanela=NUM_JANELA
    for key, value in dic_sentences.items():
        #print('value:', value)
        entidades = value[1]
        if len(entidades)==0:
            continue
        for entidade in entidades:
            #print('entidade:', entidade)
            label = entidade[2]
            indiceEntidade1=entidade[1][0]
            indiceEntidade2=entidade[1][-1]
            vizinhosAntes = list()
            vizinhosDepois = list()
            #print('indiceEntidade:', indiceEntidade)
            for tokens in value[0]:
                indice=tokens[1]
                #print('token: {}, indice: {}'.format(tokens[0], indice))
                if indice+4>=indiceEntidade1 and indice<indiceEntidade1:
                    vizinhosAntes.append(tokens[0])
                if indice-4<=indiceEntidade2 and indice>indiceEntidade2:
                    vizinhosDepois.append(tokens[0])
            lista.append([entidade[0], vizinhosAntes, vizinhosDepois, label])
        listaSentencas.append(lista)
        lista=list()
    return listaSentencas

In [204]:
lista = getListaPositiva(dic_sentencesTest)
lista[:2]

[[['FA',
   ['há', '5', 'anos', 'por'],
   [',', 'uso', 'de', 'marevan'],
   'Problema'],
  ['marevan 5mg',
   ['FA', ',', 'uso', 'de'],
   ['1', 'x', 'ao', 'dia'],
   'Tratamento']],
 [['Comorbidades', [], [':', 'DM', 'há', '10'], 'Problema'],
  ['DM', ['Comorbidades', ':'], ['há', '10', 'anos', 'em'], 'Problema'],
  ['metformina 850mg',
   ['anos', 'em', 'uso', 'de'],
   ['3', 'cp', '/', 'dia'],
   'Tratamento'],
  ['acarbose', ['cp', '/', 'dia', ','], ['1', 'cp', '/', 'dia'], 'Tratamento'],
  ['glicazida 60mg',
   ['cp', '/', 'dia', 'e'],
   ['2', 'cp', '/', 'dia'],
   'Tratamento'],
  ['insulina', ['cp', '/', 'dia', 'e'], ['(', '24', '-', '0'], 'Tratamento']]]

In [205]:
listaDev = getListaPositiva(dic_sentencesDev)
listaDev[:2]

[[['HAS', [], [',', 'ICC', ',', 'nega'], 'Problema'],
  ['ICC', ['HAS', ','], [',', 'nega', 'DM', '.'], 'Problema'],
  ['DM', [',', 'ICC', ',', 'nega'], ['.'], 'Problema']],
 [['carvedilol', [], ['1', 'cp', '12', '/'], 'Tratamento'],
  ['furosemida 20mg',
   ['12', '/', '12', ','],
   ['2', 'cp', 'de', '12'],
   'Tratamento'],
  ['sinvastatina',
   ['12', '/', '12', ','],
   ['1cp', 'a', 'noite', ','],
   'Tratamento'],
  ['AAS 100mg',
   ['1cp', 'a', 'noite', ','],
   ['apos', 'almoço', 'e', 'Omeprazol'],
   'Tratamento'],
  ['Omeprazol 20mg',
   ['100mg', 'apos', 'almoço', 'e'],
   ['1', 'xdia', '.'],
   'Tratamento']]]

In [82]:
dicPostagger['cardiaco']

'ADJ'

In [98]:
dicPostagger = load_obj('../spanclassification/obj/dic_postagger')
def tipoPostaggerTokens(token, dicPostagger):
    postagger = 'N' # na duvida é N
    if token.lower() in dicPostagger.keys():
        postagger = dicPostagger.get(token.lower())
    return postagger
tipoPostaggerTokens('coração', dicPostagger)

Load obj em:  obj/../spanclassification/obj/dic_postagger.pkl


'N'

In [213]:
def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        print(line)
        print(num)
        raise
    return word2cluster


def word2features(sent, word2cluster, dicPostagger):
    try:
        features = list()
        entidades = sent[0]
        for entidade in entidades.split():
            postag = tipoPostaggerTokens(entidade, dicPostagger)
            features.extend([
            'bias',
            'word.lower=' + entidade.lower(),
            'word[-3:]=' + entidade[-3:],
            'word[-2:]=' + entidade[-2:],
            'word.isupper=%s' % entidade.isupper(),
            'word.istitle=%s' % entidade.istitle(),
            'word.isdigit=%s' % entidade.isdigit(),
            'word.cluster=%s' % word2cluster[entidade.lower()] if entidade.lower() in word2cluster else "0",
            'postag=' + postag
            ])
        # palavras anteriores
        vizinhosAntes = sent[1] 
        if len(vizinhosAntes)>0:
            for num, vizinhoAntes in enumerate(vizinhosAntes):
                word1 = vizinhoAntes
                postag1 =  tipoPostaggerTokens(vizinhoAntes, dicPostagger)
                features.extend([
                    '-'+str(num+1)+':word.lower=' + word1.lower(),
                    '-'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '-'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '-'+str(num+1)+':postag=' + postag1
                ])
        else:
            features.append('BOS')

        # próximas palavras
        vizinhosDepois = sent[2]
        if len(vizinhosDepois)>0:
            for num, vizinhoDepois in enumerate(vizinhosDepois):
                word1 = vizinhoDepois
                postag1 =  tipoPostaggerTokens(vizinhoDepois, dicPostagger)
                features.extend([
                    '+'+str(num+1)+':word.lower=' + word1.lower(),
                    '+'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '+'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '+'+str(num+1)+':postag=' + postag1
                ])
        else:
            features.append('EOS')
    except:
        print('sent:', sent)
        raise
    return features


def sent2features(lista, word2cluster, dicPostagger):
    #return word2features(lista, word2cluster, dicPostagger)
    return [word2features(lista[i], word2cluster, dicPostagger) for i in range(len(lista))]

def sent2labels(lista):
    #return [label for _, _, _, label in lista]
    return [label for _, _, _, label in lista]


#def sent2tokens(sent):
#    return [token for token, postag, label in sent]

word2cluster = read_clusters(r"clusters/cluster-50.tsv")

In [214]:
print('dor:',word2cluster['dor'])
print('esquerdo:',word2cluster['esquerdo'])
print('direito:',word2cluster['direito'])
print('peito:',word2cluster['peito'])
print('no:',word2cluster['no'])
print('na:',word2cluster['na'])
print('coração:',word2cluster['coração'])
print('válvula:',word2cluster['válvula'])
print('cardíaca:',word2cluster['cardíaca'])
print('miocárdica:',word2cluster['miocárdica'])
print('mitral:',word2cluster['mitral'])
print('mmii:',word2cluster['mmii'])
print('coronária:',word2cluster['coronária'])

dor: 25
esquerdo: 11
direito: 11
peito: 31
no: 24
na: 24
coração: 11
válvula: 29
cardíaca: 9
miocárdica: 46
mitral: 41
mmii: 7
coronária: 5


In [215]:
sent2features(lista[0], word2cluster, dicPostagger)[0]

['bias',
 'word.lower=fa',
 'word[-3:]=FA',
 'word[-2:]=FA',
 'word.isupper=True',
 'word.istitle=False',
 'word.isdigit=False',
 'word.cluster=8',
 'postag=N',
 '-1:word.lower=há',
 '-1:word.istitle=False',
 '-1:word.isupper=False',
 '-1:postag=V',
 '-2:word.lower=5',
 '-2:word.istitle=False',
 '-2:word.isupper=False',
 '-2:postag=NUM',
 '-3:word.lower=anos',
 '-3:word.istitle=False',
 '-3:word.isupper=False',
 '-3:postag=N',
 '-4:word.lower=por',
 '-4:word.istitle=False',
 '-4:word.isupper=False',
 '-4:postag=PREP',
 '+1:word.lower=,',
 '+1:word.istitle=False',
 '+1:word.isupper=False',
 '+1:postag=PU',
 '+2:word.lower=uso',
 '+2:word.istitle=False',
 '+2:word.isupper=False',
 '+2:postag=N',
 '+3:word.lower=de',
 '+3:word.istitle=False',
 '+3:word.isupper=False',
 '+3:postag=PREP',
 '+4:word.lower=marevan',
 '+4:word.istitle=False',
 '+4:word.isupper=False',
 '+4:postag=N']

In [216]:
#X_train = word2features(lista, word2cluster, dicPostagger)
#X_train = [sent2features(s, word2cluster) for s in train_sents]
X_train = [sent2features(s, word2cluster, dicPostagger) for s in lista]
#y_train = [sent2labels(s) for s in train_sents]


In [217]:
print(X_train[0])

[['bias', 'word.lower=fa', 'word[-3:]=FA', 'word[-2:]=FA', 'word.isupper=True', 'word.istitle=False', 'word.isdigit=False', 'word.cluster=8', 'postag=N', '-1:word.lower=há', '-1:word.istitle=False', '-1:word.isupper=False', '-1:postag=V', '-2:word.lower=5', '-2:word.istitle=False', '-2:word.isupper=False', '-2:postag=NUM', '-3:word.lower=anos', '-3:word.istitle=False', '-3:word.isupper=False', '-3:postag=N', '-4:word.lower=por', '-4:word.istitle=False', '-4:word.isupper=False', '-4:postag=PREP', '+1:word.lower=,', '+1:word.istitle=False', '+1:word.isupper=False', '+1:postag=PU', '+2:word.lower=uso', '+2:word.istitle=False', '+2:word.isupper=False', '+2:postag=N', '+3:word.lower=de', '+3:word.istitle=False', '+3:word.isupper=False', '+3:postag=PREP', '+4:word.lower=marevan', '+4:word.istitle=False', '+4:word.isupper=False', '+4:postag=N'], ['bias', 'word.lower=marevan', 'word[-3:]=van', 'word[-2:]=an', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=False', 'word.cluster=17', 

In [219]:
#y_train = sent2labels(lista)
y_train =[sent2labels(s) for s in lista]

In [220]:
X_dev = [sent2features(s, word2cluster, dicPostagger) for s in listaDev]
y_dev = [sent2labels(s) for s in listaDev]

In [221]:
y_train[:10]

[['Problema', 'Tratamento'],
 ['Problema',
  'Problema',
  'Tratamento',
  'Tratamento',
  'Tratamento',
  'Tratamento'],
 ['Problema', 'Tratamento', 'Tratamento', 'Tratamento', 'Tratamento'],
 ['Problema', 'Tratamento', 'Tratamento'],
 ['Problema', 'Problema', 'Problema', 'Problema', 'Problema'],
 ['Problema', 'Teste', 'Teste', 'Teste', 'Problema'],
 ['Problema', 'Problema'],
 ['Anatomia', 'Problema'],
 ['Anatomia', 'Problema', 'Anatomia'],
 ['Teste', 'Teste']]

In [222]:
print(len(lista))
print(len(X_train))
print(len(y_train))

399
399
399


In [225]:
import time

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

loading training data to CRFsuite: 100%|██████████| 399/399 [00:00<00:00, 7124.82it/s]


loading dev data to CRFsuite: 100%|██████████| 327/327 [00:00<00:00, 12110.49it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 5102
Seconds required: 0.030

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.01  loss=1286.91  active=4999  precision=0.088  recall=0.250  F1=0.130  Acc(item/seq)=0.353 0.315  feature_norm=0.25
Iter 2   time=0.01  loss=1231.02  active=4961  precision=0.307  recall=0.280  F1=0.187  Acc(item/seq)=0.380 0.318  feature_norm=0.28
Iter 3   time=0.01  loss=1053.20  active=4863  precision=0.727  recall=0.557  F1=0.518  Acc(item/seq)=0.612 0.498  feature_norm=0.73
Iter 4   time=0.00  loss=899.16   active=5000  precision=0.710  recall=0.661  F1=0.669  Acc(item/seq)=0.701 0.569  feature_norm=1.21
Iter 5   time=0.00  loss=707.84   active=4960  pre

Iter 90  time=0.00  loss=119.00   active=1646  precision=0.872  recall=0.871  F1=0.869  Acc(item/seq)=0.872 0.807  feature_norm=18.20
Iter 91  time=0.00  loss=118.99   active=1645  precision=0.872  recall=0.871  F1=0.869  Acc(item/seq)=0.872 0.807  feature_norm=18.21
Iter 92  time=0.00  loss=118.98   active=1642  precision=0.872  recall=0.871  F1=0.869  Acc(item/seq)=0.872 0.807  feature_norm=18.20
Iter 93  time=0.00  loss=118.98   active=1642  precision=0.872  recall=0.871  F1=0.869  Acc(item/seq)=0.872 0.807  feature_norm=18.21
Iter 94  time=0.00  loss=118.97   active=1641  precision=0.872  recall=0.871  F1=0.869  Acc(item/seq)=0.872 0.807  feature_norm=18.21
Iter 95  time=0.00  loss=118.97   active=1641  precision=0.872  recall=0.871  F1=0.869  Acc(item/seq)=0.872 0.807  feature_norm=18.21
Iter 96  time=0.00  loss=118.96   active=1640  precision=0.872  recall=0.871  F1=0.869  Acc(item/seq)=0.872 0.807  feature_norm=18.21
Iter 97  time=0.00  loss=118.95   active=1642  precision=0.870

In [226]:
!pip install eli5
import eli5

eli5.show_weights(crf, top=30)

ModuleNotFoundError: No module named 'eli5'

In [39]:
dic_predictions_results_ner = load_obj(r'../spanclassification/obj/dic_predictions_results_ner_800')

Load obj em:  obj/../spanclassification/obj/dic_predictions_results_ner_800.pkl


In [46]:
dic_predictions_results_ner[42]

[[['Paciente', 0],
  ['em', 1],
  ['tratamento', 2],
  ['para', 3],
  ['ICC', 4],
  ['diastólica', 5],
  ['há', 6],
  ['cerca', 7],
  ['de', 8],
  ['5', 9],
  ['a', 10],
  ['com', 11],
  ['melhora', 12],
  ['importante', 13],
  ['dos', 14],
  ['sintomas', 15],
  ['após', 16],
  ['início', 17],
  ['do', 18],
  ['tratamento', 19],
  ['.', 20]],
 [['ICC diastólica', [4, 5], 'Problema'], ['tratamento', [19], 'Tratamento']]]

In [86]:
# para corpus GENIA, ideal pq pega as descontinuas
# no nestedclin, as descontinuas
def powersetGenia(s):
    x = len(s)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield [ss for mask, ss in zip(masks, s) if i & mask]
        
def powerset(entidade):    
    lista=list()
    for i in range(1, len(entidade)+1):
        #lista.append(entidade[i-1])
        for j in range(len(entidade) - i + 1):
            lista.append(entidade[j:j + i])
    return lista
        
#print(list(powerset([4, 5, 6])))
print(powerset(['dor', 'no', 'peito']))


[['dor'], ['no'], ['peito'], ['dor', 'no'], ['no', 'peito'], ['dor', 'no', 'peito']]


In [22]:
#samplecorpusformat = [[['POI', 'B-Abbreviation'],  ['DE', 'O'],  ['LAVAGEM', 'O'],  ['+', 'O'],  ['CURETA', 'O'],  ['DE', 'O'],  ['TECIDO', 'O'],  ['NECROTICO', 'O'],  ['.', 'O']], [['18', 'O'],  [':', 'O'],  ['00', 'O'],  [':', 'O'],  ['PACIENTE', 'O'],  ['RETORNOU', 'O'],  ['DO', 'O'],  ['CC', 'B-Abbreviation'],  ['LUCIDO', 'O'], [',', 'O'],  ['ORIENTADO', 'O'],  [',', 'O'],  ['COMUNICATIVO', 'O'],  [';', 'O'], ['MANTEM', 'O'],  ['AVP', 'B-Abbreviation'],  ['COM', 'O'],  ['STP', 'B-Abbreviation'], ['.', 'O']]]

# corpus:
# gravar os indices dos tokens
# para cada entidade, gerar a combinação possivel de tokens e pegar as features

